In [1]:
%load_ext sagemaker_studio_analytics_extension.magics
%sm_analytics emr-serverless connect --application-id 00fo38nsirh68609 --language python --emr-execution-role-arn arn:aws:iam::442426877041:role/service-role/AmazonEMRStudio_RuntimeRole_1731016215290

Waiting for EMR Serverless application state to become STARTED
Waiting for EMR Serverless application state to become STARTED
Initiating EMR Serverless connection..
Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,00fo66516l1ehh0a,pyspark,idle,"<a target=""_blank"" href=""https://j-00fo66516l1ehh0a.dashboard.emr-serverless.us-east-1.amazonaws.com/?authToken=eDAxxzO8FqjpZ2MC1SMfS3iYn0oCmXp8rAgTE9uk9v9v6P84S0jMh1WNQUa_61I2DhE3QhgC_wDr0Hxq1KUkeJDW20ubaMi1F-TEoOF_EE6jddOqxq_9tfj1HSLAE49r72R9X9zKaKyo2JTQWCjhvOhaI-sB8kGtkUQsASXZZWt3Wpn3nvsD0e8dvMIn_ZiBPssMYrAtnqJ61UAfuNCXAUSOHZ6SluJxnBsFM8sKqRQ2dnlCutNQjdabGUNspEEMXiZiQYaV7Qz8FY6mKuoJCwrpzEf_OTAC3UQ9kop1hbLo7Nlzp6frPp64nV8wlykimXin2RpAdVQszO84QlZSWtm2fdB3iI62yh1mJja3QcT-q18qIyBzIFC6j8VmtdLR4Ls2Jzvv70-yQLL0X-_GURDPQAs0Hb2Fm4ZMms9QBkmrYg677KfUEVANVkb3eCQMTVnRCbv9B4sXdTZL8SXxa6wWVnF103didVgaCBaeRzi4sI518BVsSOyRwYAsQIH0yoKvK77y7vlqI6z4NyzBd-N_CZayJ0zuA2IuOrl6iw6WTnO3anezhW2ciAmKI9Fi9JHehwoeKoItmFOcTkKK-HvKTZnlZwE60eyAcfpF2Pm-DtZrSZqpNWKA0KSUvA1Dp7p4NS-OeucJz57MoCJZXbgyTqbyyThwJyByIgjQ62s1q1Yx02ufeXtGRvMD_97sLdEKqGwpHJtuakGeCTqeJ_wNUD7whFSMgJMr-ZWWSdJyEN_l7LsK8NsWYA9YsdvPj1jYoAKDuqt3yxxODvsiws6fInmWiXETXcvi2aW1GaOcsE9htTrWs_Fq0RvmouuyFXyIWh-O0nfjgabFGlmrEjDQ4Ghkqhs2xiRouHjm6if3otP_ZxyNws-RgOJg9Y5gamDla2-WyqAhR2AmZppZHKf-lI7Sd7wHW0N7juWY4UGPz7i0FJufgn4vcYu9iSlSHiY.eyJraWQiOiJBUUlCQUhnTVJ1MlZyR2FFajR3bTc2dEZqMThGMUsyVFdZbkk4WXpxS0lRamg5YTZHZ0V4blFlbnA5SXNVYTQ2U1k2cFh6QUZBQUFBb2pDQm53WUpLb1pJaHZjTkFRY0dvSUdSTUlHT0FnRUFNSUdJQmdrcWhraUc5dzBCQndFd0hnWUpZSVpJQVdVREJBRXVNQkVFREZBenROdVZzVkNEd3JDOWR3SUJFSUJibEdxWHJENDlZOVJyV3Y2allDV0RoR1RkTWFkSUV5S0ExNFo2NkZoQjZBbXdveE5XbEFVRVdhWU96d0ZGV3VrRmNnQ2JaOE1hdW1sMlhNOVFkem5kVnNXVTFGQ1lvbEhOUTMvY2hPbmMrM1JtRENkUVNGRUJNSmVVZUE9PSJ9"">Link","<a target=""_blank"" href=""https://j-00fo66516l1ehh0a.dashboard.emr-serverless.us-east-1.amazonaws.com/logs/SPARK_DRIVER/stderr.gz?authToken=eDAx4Gc3osnklVYsiZzIYKIZu1ZNTbOdQ0R_Tc6iiU1hkorTKeR-9lAYI4_IzyV_qXz8NbO3O5DVJJ3lKEUfWCOD8KFearIWHp9jrJhlSEFXLzy_diIZEohjvnWHrHOq2gcXfWT1WLOgDEQ39_NVqaP8SP-kGtACbaNzXUe0afE4fPGAM5x3kwoIqbeq5UEZM2Bw68lRncA8Yanwb1AbgMfy1CXn40zErpf2JknuHgEaTNgBuBMpf6aHybNoyrePUNnT8Odkrej2mQ_qo3ZaP8Lh_m6hamiWxrAUofpjccMRBmKGWD4dc3UzlPnujWsmGfoji6OpexkIT47gAjxgkI6gh0f9Tg7oy2gRanwyBUV2qvxd6WAy4awY9lnJdwTeLzc1b23oCtS5fGIpBt2GjxrMfLXaBdOGyFeTwXnH81WqYMDkM59qeua-hwDuSmQ1wGr6exoRQcFGns3vQx6LZRfuhY5CjXRafah-3lcYxxliUaJHDW3eOu0SklDXqeLGV5W3932Ymcq9l6Q0A97oEBiqmy577BIMSxYNldzrdHhHeiZc4NHQVUvgbYoVhgNVmJS04yc-d-mA0gUewMdWFfB73z3_vH9cAuIo2aY9FS_QKDN3WIAAuf0mcbzDSR2irF3eadAJTxTL1vYeepnGwp7T6aAK42-9H3almwwhLBcRKVHJoV5j1Eo4vczYoEl9pO8H2cSTP4utREORsujs8B_4E6iZzrHYogNenn92xmFHyyA-ZYuoAjzXAG2aWzVTu1LZWPM0BAWWG5IQRRf_qRhfZBzv5U-L2GMTnDAR1xYC3M0WO6ksNAlj1uRhq3J5y0wbG7_UPvIO2OGMIeUN89lZMNNOBlBfCsRuCKmrxlKCeOvekM-zWbq1YhcSeXGwfcM1lQAKAnJCzJZ-CfVOqE6j6dffR9cmIcxCcYsXvndDQuB1vwUiPFXdqRCdpzKXEUw.eyJraWQiOiJBUUlCQUhnTVJ1MlZyR2FFajR3bTc2dEZqMThGMUsyVFdZbkk4WXpxS0lRamg5YTZHZ0V4blFlbnA5SXNVYTQ2U1k2cFh6QUZBQUFBb2pDQm53WUpLb1pJaHZjTkFRY0dvSUdSTUlHT0FnRUFNSUdJQmdrcWhraUc5dzBCQndFd0hnWUpZSVpJQVdVREJBRXVNQkVFREZBenROdVZzVkNEd3JDOWR3SUJFSUJibEdxWHJENDlZOVJyV3Y2allDV0RoR1RkTWFkSUV5S0ExNFo2NkZoQjZBbXdveE5XbEFVRVdhWU96d0ZGV3VrRmNnQ2JaOE1hdW1sMlhNOVFkem5kVnNXVTFGQ1lvbEhOUTMvY2hPbmMrM1JtRENkUVNGRUJNSmVVZUE9PSJ9"">Link",None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


no need of: from pyspark.sql import SparkSession

it happens above automatically

In [2]:
import boto3
import json
from urllib.parse import quote_plus
from pyspark.sql.functions import col

# Fetch the secret
secretsmanager_client = boto3.client('secretsmanager', region_name='us-east-1')  # Specify your region
secret_response = secretsmanager_client.get_secret_value(SecretId="redshift!default-namespace-admin")

# Parse the SecretString
credentials = json.loads(secret_response['SecretString'])

# Extract username and password
username = credentials['username']
password = credentials['password']

encoded_password = quote_plus(password)

# Configuration options
jdbc_url = f"jdbc:redshift://default-workgroup.442426877041.us-east-1.redshift-serverless.amazonaws.com:5439/re-pricing-db?user={username}&password={encoded_password}"
aws_iam_role_arn = "arn:aws:iam::442426877041:role/service-role/AmazonRedshift-CommandsAccessRole-20241009T164003"

s3_temp_dir = "s3://apartment-pricing/temp/"
output_s3_path = "s3://apartment-pricing/preprocessed/data/parquet/"

# SQL query to execute
query = """SELECT * FROM "train-db".rental_pricing_table"""

# Read from Redshift
df = spark.read \
    .format("io.github.spark_redshift_community.spark.redshift") \
    .option("url", jdbc_url) \
    .option("query", query) \
    .option("tempdir", s3_temp_dir) \
    .option("aws_iam_role", aws_iam_role_arn) \
    .load()


df = df.withColumn("walkscore", col("walkscore") / 10)

# Save preprocessed data to S3
df.write.parquet(output_s3_path, mode="overwrite")
print(f"Preprocessed data written to {output_s3_path}")


df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Preprocessed data written to s3://apartment-pricing/preprocessed/data/parquet/
+-------------+--------+------------+----------+------------+----------+-----------------------+-------------------+------------+---------+
|squarefootage|numrooms|numbathrooms|hasbalcony|hasgymaccess|hasparking|neighborhoodsafetyindex|          walkscore|schoolrating|     rent|
+-------------+--------+------------+----------+------------+----------+-----------------------+-------------------+------------+---------+
|          400|       4|           2|         0|           0|         1|              8.9200001| 6.4000000000000004|         2.3|3249.9299|
|          400|       4|           2|         0|           0|         1|              8.9200001| 6.4000000000000004|         2.3|3249.9299|
|          400|       3|           3|         0|           0|         0|                   1.72|                7.5|         1.9|2741.3301|
|          400|       4|           3|         1|           0|         1|         

In [3]:
df.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

655360000

In [4]:
from pyspark.sql.functions import avg

mean_walkscore = df.select(avg("walkscore")).collect()[0][0]
print(f"Mean of walkscore: {mean_walkscore}")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Mean of walkscore: 4.944520000008283